Import Libraries

In [47]:
import pandas as pd
import numpy as np
import os
import yfinance as yf


Currency Codes

In [48]:
country_currency_codes_table = {
    'Burundi': 'BIF',
    'Benin': 'XOF',
    'Burkina Faso': 'XOF',
    'Central African Republic': 'XAF',
    "Côte d'Ivoire": 'XOF',
    'Cameroon': 'XAF',
    'Democratic Republic of the Congo': 'CDF',
    'Republic of Congo': 'XAF',
    'Comoros': 'KMF',
    'Eritrea': 'ERN',
    'Ethiopia': 'ETB',
    'Ghana': 'GHS',
    'Guinea': 'GNF',
    'The Gambia': 'GMD',
    'Guinea-Bissau': 'XOF',
    'Liberia': 'LRD',
    'Madagascar': 'MGA',
    'Mali': 'XOF',
    'Mozambique': 'MZN',
    'Mauritania': 'MRU',
    'Malawi': 'MWK',
    'Niger': 'XOF',
    'Rwanda': 'RWF',
    'Sudan': 'SDG',
    'Senegal': 'XOF',
    'Sierra Leone': 'SLL',
    'Somalia': 'SOS',
    'Sao Tomee and Principe': 'STN',
    'Chad': 'XAF',
    'Togo': 'XOF',
    'Tanzania': 'TZS',
    'Uganda': 'UGX',
    'Zambia': 'ZMW'
}


Read Data

In [49]:
#read in stata dataset as dataframe "country_code_df"
country_code_df = pd.read_stata("codesffactors_Mar23_2019.dta") 
country_code_df.replace('--', np.nan, inplace=True) #replace all entries that are '--' with NaN data value

country_code_df.head()

,countryname,wbctry,wbcode,wbregion,wbigroup2017,wblcat2017,hipc,emu,cemac,waemu,imfcode,imfweocode,imfweoiso,imfweoctry,region
0,Aruba,Aruba,ABW,Latin America & Caribbean,High income,NaN,0,0,0,0,NaN,314.0,ABW,Aruba,LAC
1,Afghanistan,Afghanistan,AFG,South Asia,Low income,IDA,1,0,0,0,512.0,512.0,AFG,Afghanistan,SAS
2,Angola,Angola,AGO,Sub-Saharan Africa,Lower middle income,IBRD,0,0,0,0,614.0,614.0,AGO,Angola,SSA
3,Anguilla,NaN,AIA,Latin America & Caribbean,NaN,NaN,0,0,0,0,312.0,NaN,NaN,NaN,LAC
4,Albania,Albania,ALB,Europe & Central Asia,Upper middle income,IBRD,0,0,0,0,914.0,914.0,ALB,Albania,ECA


In [50]:
region_classification_ls = country_code_df['wbregion'].unique() #get a list of region classifications
income_classification_ls = country_code_df['wbigroup2017'].unique() #get a list of income classifications
income_classification_ls = [x for x in income_classification_ls if not pd.isna(x)] # remove '--' from income_classification_ls

print(f"The region classifications are: {', '.join(country for country in region_classification_ls)}")
print(f"The income classifications are: {', '.join(country for country in income_classification_ls)}")

The region classifications are: Latin America & Caribbean, South Asia, Sub-Saharan Africa, Europe & Central Asia, Middle East & North Africa, East Asia & Pacific, North America
The income classifications are: High income, Low income, Lower middle income, Upper middle income


In [51]:
ssa_hipc_countries = country_code_df[(country_code_df['hipc'] == 1) & (country_code_df['wbregion'] == 'Sub-Saharan Africa')]
ssa_hipc_countries_ls = ssa_hipc_countries['imfweoctry'].to_list()

ssa_hipc_countries_ls

['Burundi',
 'Benin',
 'Burkina Faso',
 'Central African Republic',
 "Côte d'Ivoire",
 'Cameroon',
 'Democratic Republic of the Congo',
 'Republic of Congo',
 'Comoros',
 'Eritrea',
 'Ethiopia',
 'Ghana',
 'Guinea',
 'The Gambia',
 'Guinea-Bissau',
 'Liberia',
 'Madagascar',
 'Mali',
 'Mozambique',
 'Mauritania',
 'Malawi',
 'Niger',
 'Rwanda',
 'Sudan',
 'Senegal',
 'Sierra Leone',
 'Somalia',
 'São Tomé and Príncipe',
 'Chad',
 'Togo',
 'Tanzania',
 'Uganda',
 'Zambia']

Read in IMF's Government Finance Statistics

In [52]:
master_table = {}
folder_path = "revenue-data"

# Loop that goes through all files 'file_name' in the folder 'folder_path'
for file_name in os.listdir(folder_path):
  if file_name.endswith('.xlsx') and not file_name.startswith('~$'): #check to see if is spreadsheet .xlsx format
    raw_df = pd.read_excel(folder_path+ "/" + file_name) #read in .xlsx as dataframe
    master_table[file_name[:-5]] = raw_df #store dataframe as an entry in the master_table hash table

# master_table["Central African Republic"]
for key in master_table:
  print(master_table[key])

                                           Unnamed: 0  \
0                          Detailed Revenue Breakdown   
1                                      Country: Benin   
2                Sector: Budgetary central government   
3                             Unit: Domestic currency   
4                                     Scale: Billions   
5                                                 NaN   
6                                                 NaN   
7                                             Revenue   
8                                               Taxes   
9           Taxes on income, profits, & capital gains   
10  Taxes on income, profits, & capital gains: ind...   
11  Taxes on income, profits, & capital gains: cor...   
12   Taxes on income, profits, & capital gains: other   
13                       Taxes on payroll & workforce   
14                                  Taxes on property   
15                          Taxes on goods & services   
16                  General tax

Filtering

In [53]:
#REMOVE "JUNK" ROWS AND CONVERT TO CORRECT SCALE (MILLION/BILLIONS)
for key in master_table: #loop through table (where key is the country name) (master_table[key] is the dataframe stored in the master_table indexed by key)
  scale = master_table[key].iloc[4,0] #get if values have scale of billions or millions
  
  # Drop rows 0 to 5, reformatting indexes and column headers
  master_table[key] = master_table[key].drop([0, 1, 2, 3, 4, 5]) 
  master_table[key] = master_table[key].reset_index(drop=True) 
  master_table[key].columns = master_table[key].iloc[0]
  master_table[key] = master_table[key][1:]
  master_table[key] = master_table[key].set_index(master_table[key].columns[0])
  master_table[key].columns = [int(float(col)) for col in master_table[key].columns]
  # master_table[key] = master_table[key].fillna(0)

  if scale == "Scale: Billions":
    master_table[key] = master_table[key]* (10 ** 9) #multiple every value in dataframe by 1e9 if scale is billion
  elif scale == "Scale: Millions":
    master_table[key] = master_table[key]* (10 ** 6) #multiple every value in dataframe by 1e9 if scale is billion


#FILTER OUT YEAR COLUMNS ACCOUNTING DOESNT ADD UP
#first level: make sure 'Taxes', 'Social Contributions', 'Grants, 'Other Revenue' Sums to 'Revenue'
#second level: make sure 'Taxes on income, profits, & capital gains', 'Taxes on payroll & workforce', 'Taxes on property', 'Taxes on goods & services', 'Taxes on int trade & transactions', 'Other taxes n.e.c.' sum to 'Taxes'
first_level_ls = ['Taxes', 'Social contributions', 'Grants', 'Other revenue']
second_level_ls = ['Taxes on income, profits, & capital gains', 'Taxes on payroll & workforce', 'Taxes on property', 'Taxes on goods & services', 'Taxes on int trade & transactions', 'Other taxes n.e.c.']
                  
for key in master_table:
  years_accounting_error_ls = []
  years_available = master_table[key].columns
  for year in years_available:
    country_year_df = master_table[key][year]
    revenue = 0 
    taxes = 0
    for category in first_level_ls:
      if category in country_year_df.index:
        revenue += country_year_df[category]
      # else: 
      #   print(category)
    for category in second_level_ls:
      if category in country_year_df.index:
        taxes += country_year_df[category]
    if round(taxes, 0) != round(country_year_df['Taxes'], 0) or round(revenue, 0) != round(country_year_df['Revenue'], 0):
      print(f"{round(taxes, 0)}, {round(country_year_df['Taxes'], 0)}, {round(revenue, 0)}, {round(country_year_df['Revenue'], 0)}")

      years_accounting_error_ls.append(year)
  
  print(key)
  print(years_accounting_error_ls)
  master_table[key].drop(columns=years_accounting_error_ls, inplace=True)
 
    
    

#FILTER OUT COUNTRIES WITH LESS THAN 7 YEARS OF DATA
less_than_seven_years_data_countries = [] #create list to store countries with less than 7 years of historial data
for key in master_table: #iterate through master_table
  num_columns = master_table[key].shape[1] #count the number of columns (each column represents a year)
  if num_columns < 7: #if less than 7 columns/year, add to list to delete
    less_than_seven_years_data_countries.append(key)

print('remove')
for country in less_than_seven_years_data_countries:
  print(country)
  del master_table[country] #delete countries with less than 7 years from data table

696000000000.0, 696000000000.0, 803100000000.0, 803000000000.0
Benin
[2013]
nan, nan, nan, 1658980000000.0
2102982357155.0, 2102982357155.0, 2538050621251.0, 2538046787551.0
Senegal
[2013, 2018]
1765582000000.0, 1765580000000.0, 2099357000000.0, 2099360000000.0
1309245000000.0, 1309250000000.0, 1530426000000.0, 1530420000000.0
1968232000000.0, 1968230000000.0, 2325908000000.0, 2325910000000.0
2246232795311.0, 2246230000000.0, 2675157000000.0, 2675160000000.0
Côte d'Ivoire
[2010, 2011, 2012, 2013]
565749183905.0, 565750000000.0, 880219000000.0, 880219000000.0
695634053206.0, 695634000000.0, 1047285963615.0, 1047290000000.0
890777678539.0, 890778000000.0, 1276315000000.0, 1276310000000.0
992608703991.0, 992610000000.0, 1441635000000.0, 1441630000000.0
Burkina Faso
[2010, 2011, 2012, 2013]
nan, 89135596491.0, nan, 159856425083.0
nan, 86631395622.0, nan, 125400435089.0
nan, 104734000000.0, nan, 201430000000.0
nan, 38344670841.0, 178749663179.0, 178749663179.0
nan, 54631151708.0, 923144601

Convert Currency

In [54]:

#CONVERT LOCAL CURRENCY DENOMINATED VALUES TO USD
error_countries = []
for key in master_table:
  #get annual 2010-2019 USD to domestic currency exchange rate using yfinance library
  currency = country_currency_codes_table[key] #retrieve currency symbol from 'country_currency_codes_table'
  symbol = f"{currency}=X"
  try:
    data = yf.download(symbol, start="2010-01-01", end="2020-01-01", interval="1mo") #download monthly USD-Currency dataset from 2010-2020
    yearly_data_df = data['Close'].resample('YE').mean() #get the annual average exchange rate outputted as datafram
    yearly_data_table = yearly_data_df.to_dict() # Convert the yearly_data to a hash table with the date as the key
    year_to_exchange_rate_dict = {key.year: value for key, value in yearly_data_table.items()} # Convert the keys to only the year part
    print(year_to_exchange_rate_dict)

    #convert DataFrame values to USD
    for year in master_table[key].columns: #loop through columns in dataframe
      year_int = int(year)  # Convert year to integer to match the dictionary keys
      master_table[key][year] = master_table[key][year] / year_to_exchange_rate_dict[year_int] #divide each column's values by that year's exchange rate

  except Exception as e: #if errored out
    print(f"Could not fetch rate for {key} ({currency}): {e}")
    error_countries.append(key) #add country to error list
    pass
    
for country in error_countries:
  del master_table[country] #delete countries where currency conversion failed
 
for key in master_table:
  print(key)
  print(master_table[key])

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

{2010: 487.61083221435547, 2011: 460.5933329264323, 2012: 501.0466715494792, 2013: 492.91916910807294, 2014: 497.47583770751953, 2015: 594.4266713460287, 2016: 594.0358378092448, 2017: 577.82666015625, 2018: 557.75, 2019: 583.7949981689453}
{2010: 1422.5833231608074, 2011: 1568.4333394368489, 2012: 1563.4083455403645, 2013: 1581.8916727701824, 2014: 1632.8416646321614, 2015: 1989.9833170572917, 2016: 2136.6749674479165, 2017: 2193.6416625976562, 2018: 2273.074991861979, 2019: 2301.0916748046875}
{2010: 886.25, 2011: 910.0, 2012: 904.5833333333334, 2013: 906.6291656494141, 2014: 907.25, 2015: 910.0, 2016: 978.4496866861979, 2017: 1457.9642842610676, 2018: 1580.0, 2019: 1630.0}
{2010: 487.5991668701172, 2011: 460.59083048502606, 2012: 501.04666900634766, 2013: 492.91916910807294, 2014: 497.4808349609375, 2015: 594.4341735839844, 2016: 594.0333353678385, 2017: 578.2424977620443, 2018: 556.1933390299479, 2019: 586.7139180501302}
{2010: 2.318599979082743, 2011: 2.648924986521403, 2012: 3.52


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['STN=X']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

{2010: 487.61083221435547, 2011: 460.5933329264323, 2012: 501.0466715494792, 2013: 492.91916910807294, 2014: 497.47583770751953, 2015: 594.4266713460287, 2016: 594.0358378092448, 2017: 577.82666015625, 2018: 557.75, 2019: 583.7949981689453}
{2010: 148.9291674296061, 2011: 155.58916854858398, 2012: 247.6258316040039, 2013: 362.4624989827474, 2014: 410.5541687011719, 2015: 495.21583557128906, 2016: 700.3816680908203, 2017: 720.1374969482422, 2018: 704.3308308919271, 2019: 718.8183339436849}
Could not fetch rate for Sao Tomee and Principe (STN): Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'
{2010: 34.550000508626304, 2011: 28.579166730244953, 2012: 28.048333326975506, 2013: 29.664999802907307, 2014: 31.18499978383382, 2015: 39.24833265940348, 2016: 62.973334312438965, 2017: 62.673333168029785, 2018: 59.819166819254555, 2019: 61.90666675567627}
{2010: 2070.574991861979, 2011: 1995.8000183105469, 2012: 2154.4749755859375, 2013: 2174.90000406901

Calculate Yearly Averages

In [55]:
data_by_year_table = {
  2010: [],
  2011: [],
  2012: [],
  2013: [],
  2014: [],
  2015: [],
  2016: [],
  2017: [],
  2018: [],
  2019: [],
} #make table that stores data by year

# master_averages_ls = [] #make master averages list as place to store averages

for key in master_table:
  years_available = master_table[key].columns
  for year in data_by_year_table:
    if year in years_available:
      data_by_year_table[year].append(master_table[key][year])

yearly_averages_ls = [] #make a list of the yearly averages
for year in data_by_year_table:
  merged_year_df = pd.concat(data_by_year_table[year], axis=1, join='outer')
  #SUM OR MEAN?
  year_average_df = merged_year_df.mean(axis=1) #get the average of all the countries' averages
  yearly_averages_ls.append(year_average_df)

 
# #Get average over decade
# #loop through table (where key is the country name) (master_table[key] is the dataframe stored in the master_table indexed by key)
# for key in master_table:
#   per_category_average = master_table[key].mean(axis=1) #get the average of each category (ignoring np.NaN values) into a list
#   # per_category_average.drop('To other government units', inplace=True) #drop 'To other government units' since there's two rows named that and will error
#   master_averages_ls.append(per_category_average) #add each country average to to the master averages list



# merged_averages_df = pd.concat(master_averages_ls, axis=1, join='outer')
# average__of_averages_df = merged_averages_df.mean(axis=1) #get the average of all the countries' averages
# average__of_averages_df

for i in range(len(yearly_averages_ls)):
  print(f"YEAR: {2010+i}")
  print(yearly_averages_ls[i])

YEAR: 2010
nan
Revenue                                                    3319995057.718612
Taxes                                                      2037236310.155335
Taxes on income, profits, & capital gains                   477102928.154863
Taxes on income, profits, & capital gains: individuals      308167111.184561
Taxes on income, profits, & capital gains: corporations     205711413.175203
Taxes on income, profits, & capital gains: other             34484799.460283
Taxes on payroll & workforce                                   4234256.97596
Taxes on property                                             4680757.343167
Taxes on goods & services                                  1048752022.490061
General taxes on goods & services                           644891187.439313
Excise taxes                                                200906916.733381
Taxes on int trade & transactions                           349332272.519638
Other taxes n.e.c.                                          1

Calculate Percentages

In [56]:
yearly_averages_percentage_ls = []
for year_average_df in yearly_averages_ls:
  total_taxes = year_average_df['Taxes']
  year_average_percentage_df = year_average_df / total_taxes * 100 #take all values as a fraction of total expenditure, then convert decimal to percent, then round to two decimal places
  yearly_averages_percentage_ls.append(year_average_percentage_df)

for i in range(len(yearly_averages_percentage_ls)):
  print(f"YEAR: {2010+i}")
  print(yearly_averages_percentage_ls[i])

YEAR: 2010
nan
Revenue                                                    162.965633
Taxes                                                           100.0
Taxes on income, profits, & capital gains                   23.419125
Taxes on income, profits, & capital gains: individuals      15.126724
Taxes on income, profits, & capital gains: corporations     10.097572
Taxes on income, profits, & capital gains: other             1.692725
Taxes on payroll & workforce                                 0.207843
Taxes on property                                             0.22976
Taxes on goods & services                                   51.479154
General taxes on goods & services                           31.655198
Excise taxes                                                 9.861738
Taxes on int trade & transactions                           17.147361
Other taxes n.e.c.                                           7.516756
Social contributions                                         0.948508
Socia

Calculate Average Across Period

In [57]:
merged_percentages_df = pd.concat(yearly_averages_percentage_ls, axis=1, join='outer')
average_df = merged_percentages_df.mean(axis=1)
rounded_average_df = average_df.apply(lambda x: round(x, 2))

rounded_average_df = rounded_average_df.to_frame()
rounded_average_df

,0
nan,
Revenue,150.86
Taxes,100.00
"Taxes on income, profits, & capital gains",30.25
"Taxes on income, profits, & capital gains: individuals",15.48
"Taxes on income, profits, & capital gains: corporations",16.87
"Taxes on income, profits, & capital gains: other",2.11
Taxes on payroll & workforce,0.60
Taxes on property,0.34
Taxes on goods & services,51.76


In [58]:
#check
first_level = 0
for category in first_level_ls:
  first_level += average_df[category]
print(first_level)

second_level = 0
for category in second_level_ls:
  second_level += average_df[category]
print(second_level)

150.92413911990295
99.99999999999999
